# CIFAR-10 using Simple Convolution Neural Networks

## Importing Libraries and Packages

In [1]:
import numpy as np 
import pandas as pd 
from tensorflow import keras
from keras import Model, Input
from keras.datasets import cifar10
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential
from keras.optimizers import Adam

Using TensorFlow backend.


## Loading and Processing data

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

Before moving on let's normalize the imgae data so that it is between 0 and 1.

In [3]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [4]:
x_train.shape, y_train.shape

((50000, 32, 32, 3), (50000, 1))

As the labels are intergers let's convert them into array for simpler processing.

In [5]:
y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)

## Building and training model

In [6]:
def cnn(input_shape = (32,32,3), classes = 10) :
    X_input = Input(input_shape)
    
    #Convolution Block 1
    X = Conv2D(filters = 6, kernel_size=(3,3), activation = "relu", padding = "Same")(X_input)
    X = MaxPooling2D(pool_size = (2,2), strides = 2)(X)
    X = BatchNormalization()(X)
    
    #Convolution Block 2
    X = Conv2D(filters = 16, kernel_size=(3,3), activation = "relu", padding = "Same")(X)
    X = MaxPooling2D(pool_size = (2,2), strides = 2)(X)
    X = BatchNormalization()(X)
    
    #Convolution Block 3
    X = Conv2D(filters = 32, kernel_size=(3,3), activation = "relu", padding = "Same")(X)
    X = MaxPooling2D(pool_size = (2,2), strides = 2)(X)
    X = BatchNormalization()(X)

    
    X = Flatten()(X)
    X = Dense(32, activation = 'relu')(X)
    X = BatchNormalization(axis = 1)(X)
    X_output = Dense(10, activation = 'softmax')(X)
    
    model = Model(X_input, X_output)
    
    return model

Let's name the model and see it's structure and other important architectural details.

In [7]:
model = cnn(input_shape = (32,32,3), classes = 10)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 6)         168       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 6)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 6)         24        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 16)        880       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 16)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 16)          64  

This code below defines the important hyperparameters for training of model.<br>
We're using learning rate decay for smoother and accurate training of model.

In [8]:
batch_size = 256
epochs = 20
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.9)
optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

Compiling and fitting the model.

In [9]:
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.0)

Epoch 1/20
50000/50000 [==============================] - 17s 330us/step - loss: 1.3599 - accuracy: 0.5119
Epoch 2/20
50000/50000 [==============================] - 17s 349us/step - loss: 1.0516 - accuracy: 0.6272
Epoch 3/20
50000/50000 [==============================] - 17s 347us/step - loss: 0.9397 - accuracy: 0.6697
Epoch 4/20
50000/50000 [==============================] - 19s 373us/step - loss: 0.8787 - accuracy: 0.6902
Epoch 5/20
50000/50000 [==============================] - 19s 390us/step - loss: 0.8246 - accuracy: 0.7109
Epoch 6/20
50000/50000 [==============================] - 17s 341us/step - loss: 0.7824 - accuracy: 0.7231
Epoch 7/20
50000/50000 [==============================] - 17s 336us/step - loss: 0.7521 - accuracy: 0.7338
Epoch 8/20
50000/50000 [==============================] - 17s 335us/step - loss: 0.7298 - accuracy: 0.7438
Epoch 9/20
50000/50000 [==============================] - 17s 334us/step - loss: 0.7032 - accuracy: 0.7523
Epoch 10/20
50000/50000 [============

Checking the accuracy of the model.

In [10]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 1.0777778217315674
Test accuracy: 0.6739000082015991
